In [1]:
# 常用的引用和创建对象实体
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)

# 1, RDD 基础

Spark中RDD是一个不可变的分布式对象集合，每个RDD都被分为多个分区，这些分区运行在集群中的不同节点上。

RDD可以包含Python,Java,Scala中任意类型的对象，甚至可以包含用户自定义的对象。

两种创建方法：1，读取外部数据集    2，在驱动器程序里分发驱动器程序中的对象集合（比如list和set）

创建出来的RDD支持两种类型的操作：转化操作（transformation）和行动操作（action）

转化操作会由一个RDD生成一个新的RDD, 行动操作会对RDD计算出一个结果，并把结果返回到驱动器程序中，或者存储到外部存储系统（HDFS） 

RDD在第一次在一个行动操作中用到时，才会真正的计算，一旦Spark了解了完整的转化操作链之后，它就可以只计算求结果时真正需要的数据

默认情况下，Spark的RDD会在每次对它们进行行动操作的时重新计算

如果想在多个行动操作中重用同一个RDD,可以使用RDD.persist()(cache()也有同样的效果）让Spark把这个RDD缓存下来，不重用的RDD没必要保存下来浪费内存资源



# 2,创建RDD

最简单的方式：把一个已有的集合传给SparkContext的Parallelize()方法，但是实际应用中不常见，因为需要预先把整个数据集先放在一个机器的内存中

In [2]:
lines = sc.parallelize(["pandas","i like pandas"])

更常见的方式：从外部存储器中读取数据来创建RDD,如从文本文件中读取

In [3]:
txt = sc.textFile("Examples/README.md")

# 3,RDD操作 -- 转换操作(Transformation) + 行动操作(Action)

## 一些常见操作

### 转化操作

RDD.filter("过滤条件") ## 对RDD元素中内容进行过滤，返回一个全新的RDD

### 行动操作 每当调用一个新的行动操作时，整个RDD都会从头开始计算，避免这种低效的行为，可以将中间结果持久化

RDD.count() ## 对返回结果进行计数

RDD.take() ## 收集RDD中的一些元素

RDD.collect() ## 用于收集RDD中的所有函数，collect()不能用在大数据集上

## 惰性求值

RDD的转化操作都是**惰性求值**的，在行动操作执行之前Spark不会开始计算。RDD不是放着特定数据的数据集，而是我们通过转化操作构建的，记录着如何处理计算数据的指令列表，同理，数据的读取操作也是惰性的。


# 4, 向Spark传递函数

## Python 情况下：

1，传递较短的函数：使用lambda表达式，也可以传递顶层函数或者定义的局部函数。**切记：不要传递带有对象引用等的参数，会导致程序失败**


In [4]:
# 传递较短的函数：
# 使用lambda表达式
# word = rdd.filter(lambda s: "error" in s)
# 使用定义的函数
# def containsError(s):
#     return "error" in s
# word = rdd.filter(containsError)


# 5，常见的转换的操作和行动操作

## 基本RDD

### 针对各个元素的转化操作

RDD.map()函数接受一个函数，并把这个函数用于RDD中的每一个元素，并将函数的返回结果作为结果RDD中对应的元素值。

RDD.filter()函数接受一个函数，并将RDD中满足该函数的元素放入新的RDD中返回。

In [5]:
# 实例：计算RDD中各值的平方
nums_RDD = sc.parallelize([1,2,3,4])
squared = nums_RDD.map(lambda x: x*x).collect()
for num in squared:
    print("%i" %num)

1
4
9
16


In [6]:
# 实例，筛选出RDD中不是1的值
squared = nums_RDD.filter(lambda x: x!=1).collect()
for num in squared:
    print("%i" %num)

2
3
4


RDD.flarMap() 希望对每一个输入元素生成多个输出元素，提供给flatMap()的函数被分别用在了输入RDD的每一个元素上，返回的不是一个元素，而是一个返回值序列的迭代器。输出的RDD倒不是由迭代器组成的，我们得到的是一个包含各个迭代器且可访问的所有元素的RDD.

In [7]:
# 实例： 将行数据切分为单词
lines = sc.parallelize(["hello world","hi Worriors","for the horde, bool and lighting!!!"])
words = lines.flatMap(lambda line: line.split(" "))
contents = words.collect()
for Str in contents:
    print("%s" %Str)

hello
world
hi
Worriors
for
the
horde,
bool
and
lighting!!!


In [8]:
lines = sc.parallelize(["hello world","hi Worriors","for the horde, bool and lighting!!!"])
words = lines.map(lambda line: line.split(" "))
contents = words.collect()
print(contents)
for content in contents:
    print(content)

[['hello', 'world'], ['hi', 'Worriors'], ['for', 'the', 'horde,', 'bool', 'and', 'lighting!!!']]
['hello', 'world']
['hi', 'Worriors']
['for', 'the', 'horde,', 'bool', 'and', 'lighting!!!']


### 伪集合操作

RDD本身不是严格意义上的集合，但是支持许多集合上的操作，**但是前提是：这些操作的RDD都要求是相同的数据类型**

可以使用RDD.distinct()转化操作来生成一个只包含不同元素的新的RDD,**但是distinct()的开销很大，它需要将所有的数据通过网络进行混洗（shuffle）以确保每个元素都只有一份

#### 常见集合操作
1. union(other): 并集，返回一个包含两个RDD中所有元素的RDD
2. intersection(other): 交集，返回两个RDD中都有的元素，并除去重复的元素
3. subtract(other): 差集，返回一个由只存在与第一个RDD而不存在于第二个RDD中的所有的元素组成的RDD,也需要数据混洗
4. cartesian(other): 笛卡尔积，产生两个RDD中每个元素组合的所有可能情况，也可以对自身求笛卡尔积，此操作开销巨大


## 行动操作

常见的RDD行动操作

<img src="https://raw.githubusercontent.com/ColinsGitCode/JupyterNotebook/200ef8d5d648bc4e68b9851c3702774cc74dfc59/ipynbFiles/Materials/Basic_RDD_Action_Functions.jpg" />


In [9]:
# example of RDD.reduce()
# Example one: reduce numbers 1 to 10 by adding them up
x = sc.parallelize([1,2,3,4,5,6,7,8,9,10],2)
cSum = x.reduce(lambda accum, n: accum + n)
print(cSum)

55


In [10]:
# example of RDD.reduce()
# Example one: reduce numbers 1 to 10 by adding them up
x = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
cSum = x.reduce(lambda accum, n: accum + n)
print(cSum)

55


In [20]:
# Example of aggergate()
# get the average of nums=[1,2,3,4]
def SeqFunc(acc,value):
    return acc[0] + value, acc[1] + 1
def ComFunc(acc1,acc2):
    return acc1[0] + acc2[0], acc1[1] + acc2[1]
print(nums_RDD.collect())

container = nums_RDD.aggregate((0,0),SeqFunc,ComFunc)

print(container)
average = container[0]/float(container[1])
print(average) 

#average = nums_RDD.aggregate((0,0),
#                            (lambda acc,value: (acc[0] + value, acc[1] + 1), 
#                            lambda acc1,acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))


[1, 2, 3, 4]
(10, 4)
2.5


### reduce(). fold(), aggregate()的详细讲解可以参照如下网页

[http://www.jianshu.com/p/15739e95a46e](http://www.jianshu.com/p/15739e95a46e)